In [4]:
from torch import nn
from torchvision import models, transforms
from torch.autograd import Variable
from PIL import Image
from utils.gradcam import *
from utils.misc import Wrapper, generate_attentive_mask
from utils.dataset import CUB200_loaders
import argparse
import os

In [5]:
weight_path = './results/default/best_model.pth'
output_path = './outputs'

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

targets = [
    "conv1",
    "layer1",
    "layer2",
    "layer3",
    "layer4"
]

model = models.resnet50(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 200)
model = nn.DataParallel(model)
model = Wrapper(model, targets)
state_dict = torch.load(weight_path)
model.load_state_dict(state_dict['model'])


model = model.to(device)

print(f"Using model with validation accuracy {state_dict['best_valid_acc']*100:.2f}%")


Registered forward/backward hook on 'module.conv1'
Registered forward/backward hook on 'module.layer1'
Registered forward/backward hook on 'module.layer2'
Registered forward/backward hook on 'module.layer3'
Registered forward/backward hook on 'module.layer4'


FileNotFoundError: [Errno 2] No such file or directory: './pretrained/R50_CUB200_S448_Vanilla.pth'

In [39]:
dataset_root = '~/datasets'
crop_size = 448
batch_size = 1
num_workers = 1
train_loader, valid_loader, num_classes = CUB200_loaders(dataset_root, crop_size, batch_size, num_workers)

In [40]:
img_tensor, label = next(iter(train_loader))

label = int(label[0])

print(img_tensor.shape)
print("Label:",label)


torch.Size([1, 3, 448, 448])
Label: 63


In [41]:
tr_image = np.asarray(img_tensor[0].permute(1,2,0))
print(tr_image.shape)
tr_image = cv2.cvtColor(tr_image,cv2.COLOR_BGR2RGB)
tr_image = np.uint8(normalize(tr_image)*255)


(448, 448, 3)


In [42]:
# =========================================================================
print('='*20)
# =========================================================================

save_dir = os.path.join(output_path)

if(os.path.isdir(save_dir)):
    print('Save directory exists')
else:
    os.mkdir(save_dir)
print('Visualization saved to : ',save_dir)

# a.save(os.path.join(save_dir,'_input_image'+'.jpg'))
gcam = GradCAM(model=model.net.module)
probs, idx = gcam.forward(img_tensor.to(device))

# if idx[0] == label:
cv2.imwrite(f'./outputs/{label}_raw.png', tr_image)
print((probs*100).sum())
print(idx[0], label)


print("-"*30)
print("|  Probability  |    Class   |")
print("-"*30)

for j in range(0,len(targets)):
    gcam.backward(idx=label)
    output = gcam.generate(target_layer=targets[j])
    print(output.shape)
    # Filename : {ClassName}_gcam_{NumLayer}
    save_gradcam(save_dir+'/{}_gcam_{}.png'.format(label, targets[j]), output, tr_image)
print('[{:.5f}] {}'.format(probs[label], label))



Save directory exists
Visualization saved to :  ./outputs
tensor(100., device='cuda:0', grad_fn=<SumBackward0>)
tensor(86, device='cuda:0') 63
------------------------------
|  Probability  |    Class   |
------------------------------
(224, 224)
(112, 112)
(56, 56)
/home/ryan/.local/lib/python3.8/site-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
(28, 28)
(14, 14)
[0.00581] 63
